In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import time
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

In [3]:
driver = webdriver.Chrome()

In [5]:
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

In [7]:
import random
import time

my_mail = "xyz@gmail.com"
password = 'xyz@385'
username = "xyz"

wait = WebDriverWait(driver, 10)

time.sleep(random.uniform(1.5, 4))


input_box = wait.until(EC.visibility_of_element_located((
    By.CSS_SELECTOR, "input[id='username']"
)))
input_box.clear()
input_box.send_keys(my_mail)

time.sleep(random.uniform(1.5, 5))

password_box = wait.until(EC.visibility_of_element_located((
    By.CSS_SELECTOR, "input[id='password']"
)))
password_box.clear()
password_box.send_keys(password)

time.sleep(random.uniform(4, 6))


sign_in_button = wait.until(EC.element_to_be_clickable((
    By.CSS_SELECTOR, "button[aria-label='Sign in']"
)))
sign_in_button.click()


In [10]:
from datetime import datetime, timedelta

def relative_to_date(text):
    today = datetime.today()
    
    if 'h' in text:
        hours = int(text.replace('h', ''))
        return today - timedelta(hours=hours)
    elif 'd' in text:
        days = int(text.replace('d', ''))
        return today - timedelta(days=days)
    elif 'w' in text:
        weeks = int(text.replace('w', ''))
        return today - timedelta(weeks=weeks)
    elif 'mo' in text:
        months = int(text.replace('mo', ''))
        return today - timedelta(days=30 * months)
    elif 'y' in text:
        years = int(text.replace('y', ''))
        return today - timedelta(days=365 * years)
    else:
        return today  # fallback

print(relative_to_date("2d"))


2025-04-19 15:05:59.399243


In [13]:
blinkit_keywords = [

    # 📦 Service / Delivery Issues
    "Blinkit Complaint",
    "Blinkit late delivery",
    "Blinkit delivery issue",
    "Blinkit order not delivered",
    "Blinkit delivery delay",
    "Blinkit missed delivery",
    "Blinkit wrong item",
    "Blinkit delivery boy issue",
    "Blinkit cancelled my order",
    "Blinkit not delivering",
    "Blinkit didn't deliver",
    "Blinkit delivery scam",
    "Blinkit service complaint",

    # 💳 Refund / Payment Issues
    "Blinkit refund not received",
    "Blinkit refund delay",
    "Blinkit charged twice",
    "Blinkit payment failed",
    "Blinkit payment issue",
    "Blinkit not refunding",
    "Blinkit money not returned",
    "Blinkit deducted money",

    # 🥦 Product Quality Issues
    "Blinkit expired product",
    "Blinkit rotten vegetables",
    "Blinkit damaged item",
    "Blinkit sent stale food",
    "Blinkit poor packaging",
    "Blinkit quality issue",
    "Blinkit not fresh",

    # 👎 Negative Sentiment / Experience
    "Blinkit worst experience",
    "Blinkit terrible service",
    "Blinkit poor customer service",
    "Blinkit very bad experience",
    "Blinkit never ordering again",
    "Blinkit worst app ever",
    "Blinkit horrible service",
    "Blinkit frustrating",
    "Blinkit disappointed",
    "Blinkit cheated",
    "Blinkit scam",
    "Blinkit fraud",

    # 🧑‍💻 Customer Support Issues
    "Blinkit no response",
    "Blinkit not responding",
    "Blinkit support not working",
    "Blinkit customer care",
    "Blinkit help",
    "Blinkit complaint",
    "Blinkit issue",

    # 🔖 Hashtags (used as plain text for Twitter search)
    "#BlinkitScam",
    "#BlinkitComplaint",
    "#BoycottBlinkit",
    "#BlinkitFraud",
    "#BlinkitIssue"
]

In [18]:
import time
import random
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timedelta

all_posts = set()
post_list = []


for query in tqdm(blinkit_keywords):
    wr_query = ("%20").join(query.split())
    link = "https://www.linkedin.com/search/results/content/?datePosted=%22past-month%22&keywords=" + wr_query+ "&origin=FACETED_SEARCH&sid=GtM"


    driver.get(link)
    time.sleep(random.uniform(4, 7))
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(4, 7))
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        posts = soup.find_all('div', class_='update-components-actor__meta')  # reliable anchor
        posts_full = soup.find_all('div', attrs={"data-urn": True, "role": "article"})
    
        for post in posts_full:
            try:
                p1 = post.find('div', class_="fie-impression-container").find_all('div', recursive=False)
            except Exception as e:
                print("Skipped post - outer structure missing:", e)
                continue
    
            try:
                check_id = post.get('data-urn')
                post_link = "https://www.linkedin.com/feed/update/" + check_id
            except:
                post_link = ""
    
            if post_link in all_posts:
                continue
    
            try:
                user_profile = p1[0].find('a').get('href')
            except:
                user_profile = ""
    
            try:
                name = p1[0].find('span', class_="update-components-actor__title")\
                           .find('span', dir='ltr')\
                           .find('span', class_=None).text.strip()
            except:
                name = ""
    
            try:
                spans = p1[0].find_all("span", {"aria-hidden": "true"})
                initial_date = ""
                for span in spans:
                    txt = span.get_text(strip=True)
                    match = re.match(r"(\d+)(w|d|mo|h|y)", txt)
                    if match:
                        initial_date = match.group(0)
                        break
    
                if initial_date:
                    date_post = relative_to_date(initial_date).strftime('%Y-%m-%d')
                else:
                    date_post = ""
            except:
                date_post = ""
    
            try:
                post_text = p1[1].find('span', dir="ltr").text.strip()
            except:
                post_text = ""
    
            try:
                if post.find('button', class_="update-components-image__image-link") and \
                   post.find('button', class_="update-components-image__image-link").find('img'):
                    have_img_vid = "Yes"
                    p_stats = p1[3]
                else:
                    raise Exception
            except:
                have_img_vid = "No"
                try:
                    p_stats = p1[2]
                except:
                    p_stats = None
    
            try:
                likes = p_stats.find('ul', class_="display-flex flex-wrap")\
                               .find_all('li')[0].find('span').text.strip()
            except:
                likes = "0"
    
            try:
                comments = p_stats.find('ul', class_="display-flex flex-wrap")\
                                  .find_all('li')[1]\
                                  .find('ul', class_="display-flex flex-grow-1 max-full-width")\
                                  .find_all('li')[0].text.strip().split(" ")[0]
            except:
                comments = "0"
    
            try:
                reposts = p_stats.find('ul', class_="display-flex flex-wrap")\
                                 .find_all('li')[1]\
                                 .find('ul', class_="display-flex flex-grow-1 max-full-width")\
                                 .find_all('li')[1].text.strip().split(" ")[0]
            except:
                reposts = "0"
    
            print(check_id)
            
            post_list.append([
                name,
                user_profile,
                date_post,
                post_text,
                post_link,
                have_img_vid,
                comments,
                reposts,
                likes            
            ])
    
            all_posts.add(post_link)
            
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
        time.sleep(random.uniform(3, 5))


  0%|                                                                                           | 0/52 [00:00<?, ?it/s]

urn:li:activity:7310184893370486785
urn:li:activity:7316026186332090368
urn:li:activity:7310152479021678592
urn:li:activity:7310331688113315841
urn:li:activity:7309090879565176832
urn:li:activity:7310363610688446464
urn:li:activity:7309446314788818944
urn:li:activity:7316117585375191040
urn:li:activity:7316014588146139138
urn:li:activity:7309803735340199936
urn:li:activity:7308751676805464065
urn:li:activity:7310291731978084352
urn:li:activity:7317606918925996032
urn:li:activity:7316788598324842497
urn:li:activity:7318938193683271680
urn:li:activity:7315459717504348160
urn:li:activity:7318572217473122304
urn:li:activity:7310989643724779522
urn:li:activity:7317033804504657920
urn:li:activity:7311046050201366532
urn:li:activity:7315035063975350273
urn:li:activity:7311812636776570880
urn:li:activity:7313549101852160001
urn:li:activity:7313602227493871616
urn:li:activity:7315033330486251522
urn:li:activity:7319288529849589760
urn:li:activity:7314504669806899202
urn:li:activity:731813609555

  2%|█▌                                                                              | 1/52 [08:03<6:50:37, 483.10s/it]

urn:li:activity:7309739163367489536
urn:li:activity:7309365202876014592
urn:li:activity:7309915673990742016
urn:li:activity:7310168559966371845
urn:li:activity:7310175625955160065
urn:li:activity:7309521798755225601
urn:li:activity:7310155988160937984
urn:li:activity:7308822824566054912
urn:li:activity:7315763905480294401
urn:li:activity:7310252988479021056
urn:li:activity:7309914385974472706
urn:li:activity:7309430020744101889
urn:li:activity:7309594263602827265
urn:li:activity:7310276804206833664
urn:li:activity:7308820180141281280
urn:li:activity:7309231167117967361
urn:li:activity:7309928475493117955
urn:li:activity:7310182190510718978
urn:li:activity:7312499642485616640
urn:li:activity:7319589207788699648
urn:li:activity:7314996750795137026
urn:li:activity:7318963302800203777
urn:li:activity:7311994905474580480
urn:li:activity:7318648284170829824
urn:li:activity:7318536203203399680
urn:li:activity:7319666501811097601
urn:li:activity:7315269269955469313
urn:li:activity:731457718483

  4%|███                                                                             | 2/52 [13:05<5:14:05, 376.91s/it]

...

urn:li:activity:7309572172417507328
urn:li:activity:7309099287060111361
urn:li:activity:7309021549230006272
urn:li:activity:7309766595038392320
urn:li:activity:7310319357782306816
urn:li:activity:7309838893573410816
urn:li:activity:7317766193136680960
urn:li:activity:7317673732657803264
urn:li:activity:7312342724664246272
urn:li:activity:7317607644045680653
urn:li:activity:7312141098825457664
urn:li:activity:7318125010898862080
urn:li:activity:7318641923622084609
urn:li:activity:7315354565514616833
urn:li:activity:7314186415032922112
urn:li:activity:7311573318673883137
urn:li:activity:7312783600255389697
urn:li:activity:7320037241433554944
urn:li:activity:7315031490122137601
urn:li:activity:7319908953138544641
urn:li:activity:7317087135289733120
urn:li:activity:7311630449552830464
urn:li:activity:7312882641161129985
urn:li:activity:7318506049886392321
urn:li:activity:7310552129033707521
urn:li:activity:7319006159946874880
urn:li:activity:7319618379273617408
urn:li:activity:731406804261

 96%|███████████████████████████████████████████████████████████████████████████▉   | 50/52 [4:03:26<13:04, 392.27s/it]

urn:li:activity:7320027722091515904
urn:li:activity:7319997989844832256
urn:li:activity:7319989664713445376
urn:li:activity:7320001591065874432
urn:li:activity:7319989676331712512
urn:li:activity:7319999417854365696
urn:li:activity:7320051968071081984
urn:li:activity:7319980338380701696
urn:li:activity:7320022045369450496
urn:li:activity:7320068838782353408
urn:li:activity:7319981941749878784
urn:li:activity:7320051247900700672
urn:li:activity:7319984952790380544
urn:li:activity:7319998153359863808
urn:li:activity:7319973246777311252
urn:li:activity:7319989999683219456
urn:li:activity:7320038616817090560
urn:li:activity:7320031008764964865
urn:li:activity:7320022826130808832


 96%|███████████████████████████████████████████████████████████████████████████▉   | 50/52 [4:04:34<09:46, 293.48s/it]


KeyboardInterrupt: 

In [20]:
len(post_list)

1526

In [22]:
columns = [
    "Name",
    "Profile Link",
    "Date Posted",
    "Post Text",
    "Post Link",
    "Media",
    "Comments",
    "Reposts",
    "Likes"
]

df = pd.DataFrame(post_list, columns=columns)
df.head()

,Name,Profile Link,Date Posted,Post Text,Post Link,Media,Comments,Reposts,Likes
0,Mahima Goyal,https://www.linkedin.com/in/mahimagoyal731?min...,2025-03-31,"""Blinkit’s Big Safety Flaw: How Easy Is It to ...",https://www.linkedin.com/feed/update/urn:li:ac...,No,1,1,19
1,Ravina Desai Bandivadekar,https://www.linkedin.com/in/ravina-desai-bandi...,2025-04-14,Customer Experience Matters – A Disappointing ...,https://www.linkedin.com/feed/update/urn:li:ac...,No,5,0,30
2,CA. Rahul Gupta,https://www.linkedin.com/company/carahulguptai...,2025-03-31,𝐅𝐌𝐂𝐆 𝐃𝐢𝐬𝐭𝐫𝐢𝐛𝐮𝐭𝐨𝐫𝐬 𝐅𝐢𝐥𝐞 𝐂𝐨𝐦𝐩𝐥𝐚𝐢𝐧𝐭 𝐀𝐠𝐚𝐢𝐧𝐬𝐭 𝐐𝐮𝐢𝐜𝐤...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,2,0,10
3,CEO INDIA Magazine,https://www.linkedin.com/company/ceoindiamagaz...,2025-03-31,🚨 Blinkit Faces Backlash Over Product Weight D...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,0,0,0
4,SOUVIK BASU,https://www.linkedin.com/in/souvik-basu-120733...,2025-03-24,I am extremely disappointed with the lack of q...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,20,1,277


In [24]:
len(df)

1526

In [26]:
# df.to_csv("linkedin_posts.csv", index=False)